<a href="https://colab.research.google.com/github/cafalchio/EXvivo/blob/main/downsampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downsampling Intan dat files

Creates a copy of each channel file undersampled by 10.

Method:
IRR filter

Creates a subfolder  'downsampleLFP'

Filenames: '_sr_2500.dat'




In [29]:
#Extract file
# patoolib.extract_archive("/content/drive/MyDrive/aHuman_data_Mark/HUMAN 216.rar", 
#                          outdir="/content/drive/MyDrive/aHuman_data_Mark")

In [28]:
from tqdm.notebook import tqdm
import numpy as np
from scipy import signal
import os
from glob import glob
import mmap
import gc

In [30]:
new_path = 'downsampleLFP/'
down_rate = 50

In [33]:
for data_path in glob("/content/drive/MyDrive/aHuman_data_Mark/HUMAN */*/", recursive = True):
    # Just the first 2 channels of each slice
    if not os.path.exists(data_path + new_path):
            os.makedirs(data_path + new_path)
    print(f'Resampling channels of {data_path.split("/")[-3]} slide {data_path.split("/")[-2]}\n\n')
    for filename in tqdm(sorted(glob(data_path+'amp*.dat'))):
        new_name = data_path + new_path + os.path.basename(filename).split('.')[-2]+'_sr_500.dat'
        new_sig = np.memmap(filename, dtype='int16')*.195
        if os.path.isfile(new_name) and os.path.getsize(new_name) == \
                            os.path.getsize(filename)/down_rate:
            continue
        else:
            new_sig = signal.decimate(new_sig, down_rate, ftype='fir')          
            fp = np.memmap(new_name, dtype='int16', 
                            mode='w+', offset=0, shape=new_sig.shape)
            fp[:] = new_sig[:]
        # del new_sig, fp #delet for speed reason, if memory explodes, use gc
        # gc.collect()

Resampling channels of HUMAN 213 slide slice1_151008_165239




  0%|          | 0/64 [00:00<?, ?it/s]

Resampling channels of HUMAN 215 slide slice1_151020_133348




  0%|          | 0/64 [00:00<?, ?it/s]

Resampling channels of HUMAN 215 slide slice2_151020_185053




  0%|          | 0/64 [00:00<?, ?it/s]

Resampling channels of HUMAN 217 slide human_151110_122702




  0%|          | 0/64 [00:00<?, ?it/s]

Resampling channels of HUMAN 216 slide session1_151106_154501




  0%|          | 0/33 [00:00<?, ?it/s]